### Import Libraries and Data

In [6]:
import pandas as pd
import numpy as np
import datetime

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

C:\Users\areuve200\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\areuve200\AppData\Local\Continuum\anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


#### Code snippet for pulling data via API

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.pa.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.pa.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("hbkk-dwy3", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [31]:
#read csv
df_opioid = pd.read_csv('data//Overdose_Information_Network_Data_CY_January_2018_-_Current_Monthly_County_State_Police.csv',
                        parse_dates = ['Incident Date'])
df_opioid['Incident Dt'] = pd.to_datetime(df_opioid['Incident Date']).dt.date
df_opioid['Incident Month'] = df_opioid['Incident Date'].dt.strftime('%b')
df_opioid['Incident year'] = df_opioid['Incident Date'].dt.strftime('%Y')
df_opioid['Incident YR_MTH'] = df_opioid['Incident Date'].dt.strftime('%Y-%m')
df_opioid.columns = [x.replace(' ','_').upper() for x in df_opioid.columns]
df_opioid.sort_values(by=['INCIDENT_DATE'], inplace= True)
df_opioid.info()
df_opioid.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25880 entries, 8018 to 25282
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   INCIDENT_ID                             25880 non-null  int64         
 1   INCIDENT_DATE                           25880 non-null  datetime64[ns]
 2   INCIDENT_TIME                           25880 non-null  object        
 3   DAY                                     25880 non-null  object        
 4   INCIDENT_COUNTY_NAME                    25880 non-null  object        
 5   INCIDENT_STATE                          25880 non-null  object        
 6   VICTIM_ID                               25880 non-null  int64         
 7   GENDER_DESC                             25880 non-null  object        
 8   AGE_RANGE                               25880 non-null  object        
 9   RACE                                    25880 n

,INCIDENT_ID,INCIDENT_DATE,INCIDENT_TIME,DAY,INCIDENT_COUNTY_NAME,INCIDENT_STATE,VICTIM_ID,GENDER_DESC,AGE_RANGE,RACE,...,INCIDENT_COUNTY_LONGITUDE,VICTIM_COUNTY_FIPS_CODE,VICTIM_COUNTY_LATITUDE,VICTIM_COUNTY_LONGITUDE,INCIDENT_COUNTY_LATITUDE_AND_LONGITUDE,VICTIM_COUNTY_LATITUDE_AND_LONGITUDE,INCIDENT_DT,INCIDENT_MONTH,INCIDENT_YEAR,INCIDENT_YR_MTH
8018,37,2018-01-01,19:10:00,Monday,Carbon,Pennsylvania,13,Female,30 - 39,White,...,-75.711070,25.0,40.919367,-75.711070,"(40.919367, -75.71107)","(40.919367, -75.71107)",2018-01-01,Jan,2018,2018-01
18186,221,2018-01-01,14:34:00,Monday,Lehigh,Pennsylvania,156,Male,25 - 29,White,...,-75.600995,77.0,40.614648,-75.600995,"(40.614648, -75.600995)","(40.614648, -75.600995)",2018-01-01,Jan,2018,2018-01
14438,30,2018-01-01,13:07:00,Monday,Philadelphia,Pennsylvania,8,Male,50 - 59,White,...,-75.140236,17.0,40.335011,-75.112912,"(40.004444, -75.140236)","(40.335011, -75.112912)",2018-01-01,Jan,2018,2018-01
6838,218,2018-01-01,0:00:00,Monday,Delaware,Pennsylvania,154,Male,30 - 39,Unknown,...,-75.406277,NaN,0.000000,0.000000,"(39.916579, -75.406277)","(0.0, 0.0)",2018-01-01,Jan,2018,2018-01
5646,222,2018-01-01,14:35:00,Monday,York,Pennsylvania,158,Male,30 - 39,White,...,-76.725761,133.0,39.921925,-76.725761,"(39.921925, -76.725761)","(39.921925, -76.725761)",2018-01-01,Jan,2018,2018-01


In [58]:
df_year = df_opioid.groupby('INCIDENT_YR_MTH').size().reset_index().rename(columns={0: 'COUNT'})
df_year_gender = df_opioid.groupby(['INCIDENT_YR_MTH',
                                    'GENDER_DESC']).size().reset_index().rename(columns={0: 'COUNT'})
df_year_narcan = df_opioid.groupby(['INCIDENT_YR_MTH',
                                    'NALOXONE_ADMINISTERED']).size().reset_index().rename(columns={0: 'COUNT'})
df_year_narcan2 = df_opioid.groupby(['INCIDENT_YR_MTH',
                                    'NALOXONE_ADMINISTERED',
                                    'SURVIVE']).size().reset_index().rename(columns={0: 'COUNT'})

#filter out Unknown survival
df_year_narcan3 = df_opioid.groupby(['NALOXONE_ADMINISTERED',
                                    'SURVIVE']).size().reset_index().rename(columns={0: 'COUNT'})
df_year_narcan3 = df_year_narcan3[df_year_narcan3['SURVIVE'] != 'U']

df_year_day = df_opioid.groupby(['INCIDENT_YR_MTH',
                                 'DAY']).size().reset_index().rename(columns={0: 'COUNT'})
df_year_day['WKDAY_WKEND'] = np.where(df_year_day['DAY'].isin(['Friday','Saturday','Sunday']), 'WKEND', 'WKDAY')

#daily average number of cases
daily_avg = df_year['COUNT'].mean()

### Plotly Figures

In [33]:
#basic line graph
fig = px.line(df_year, x="INCIDENT_YR_MTH", y="COUNT", title='Overdoses in PA Since 2018')
fig.show()

In [104]:
#segment by narcan use
fig = px.line(df_year_narcan, x="INCIDENT_YR_MTH", y="COUNT", 
              color = 'NALOXONE_ADMINISTERED',
              color_discrete_map={
                "N": '#EF553B',
                "Y": "green"},
              title='Overdoses in PA Since 2018')

#vertical box
fig.add_vrect(x0="2018-07-01", x1="2019-02-01", 
              annotation_text="decline", annotation_position="top left",
              fillcolor="purple", opacity=0.25, line_width=0)
fig.show()

In [37]:
fig = px.line(df_year_narcan2, x="INCIDENT_YR_MTH", y="COUNT", 
              color = 'NALOXONE_ADMINISTERED',
              color_discrete_map={
                "N": '#EF553B',
                "Y": "green"},
              line_group="SURVIVE", 
              title='Overdoses in PA Since 2018')
fig.show()

In [108]:
#basic bar chart
fig = px.bar(df_year, x="INCIDENT_YR_MTH", y="COUNT", title='Overdoses in PA Since 2018')

#horizontal line
fig.add_hline(y=daily_avg, line_width=3, line_dash="dash", line_color="black",
              annotation_text="Daily Average", 
              annotation_position="top right")
fig.show()

In [123]:
import plotly.express as px
fig = px.bar(df_year_narcan3, x="NALOXONE_ADMINISTERED", y="COUNT", color = 'SURVIVE',
             color_discrete_map={
                "N": '#EF553B',
                "Y": "green"}, 
             #relabel
             labels=dict(SURVIVE="Survival", NALOXONE_ADMINISTERED = 'Narcan Admin', COUNT='Count'),
             #labels on bars
             text = 'COUNT',
             title='Overdoses in PA Since 2018')
fig.show()

### Plotly - Dash

In [101]:
fig = px.bar(df_year_day, x="INCIDENT_YR_MTH", y="COUNT", color = 'WKDAY_WKEND')
fig.add_hline(y=daily_avg, line_width=3, line_dash="dash", line_color="black",
              annotation_text="Daily Average", 
              annotation_position="top right")
fig.show()

In [72]:
days = df_year_day['WKDAY_WKEND'].unique()

app = JupyterDash(__name__)

# Build App
app.layout = html.Div([
    #title
    html.H2("Overdose Incidents in PA"),

    dcc.Graph(id='graph'),
    
    #dropdown
    html.Label([
        "Day of Week",
        dcc.Dropdown(
            id = "day-dropdown",
            clearable = False,
            value = days[0],
            options = [{"label": x, "value": x} for x in days]
        ),
    ])
])


@app.callback(
    Output("graph", "figure"), 
    [Input("day-dropdown", "value")]
    )

def update_bar_chart(day):
    mask = df_year_day['WKDAY_WKEND'] == day
    fig = px.bar(df_year_day[mask], x="INCIDENT_YR_MTH", y="COUNT", 
                 color = 'DAY', barmode="group")
    return fig
                            
# Run app and display result inline in the notebook
app.run_server(mode='inline')